In [9]:
assignment_url = 'https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class7.ipynb'
img_url = 'https://data.heatonresearch.com/data/t81-558/sidewalk/sidewalk{}.jpg' # img from 1 to 10

In [10]:
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import pandas as pd

model = YOLO("yolo_weights/yolov8s.pt")
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

final_list = []

def count_in_img(img_arr, img_id):
    data = {'image':[0],
            'bicycle':[0],
            'car':[0],
            'person':[0],
            'umbrella':[0],
            'motorbike':[0],
            'handbag':[0]}
    data["image"][0] = img_id
    img_arr = cv2.resize(img_arr,(0,0),None,0.50,0.50)
    result = model(img_arr)
    for r in result:
        box = r.boxes
        for b in box:
            x1,y1,x2,y2 = b.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            

            conf = round(b.conf[0].item(), 2)
            cls = b.cls[0]
            # print(int(cls))
            curClass = classNames[int(cls)]
            if curClass  == "bicycle" or curClass  == "car" or curClass  == "person" or curClass  == "umbrella" or curClass  == "motorbike" or curClass  == "handbag":
                
            # detection rectangle
                cv2.rectangle(img_arr, (x1,y1), (x2,y2), (0,0,255), 2)

                # rect info 
                h,w = cv2.getTextSize(f'{conf} {curClass }', cv2.FONT_HERSHEY_PLAIN, 2, 2)[0]
                cv2.rectangle(img_arr, (max(0,x1), max(0,y1)), (x1+h, y1-w), (0,0,0), cv2.FILLED)
                cv2.putText(img_arr, f'{conf} {classNames[int(cls)]}', (max(0,x1), max(0,y1)), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
                if curClass in data:
                    data[curClass][0] += 1
        final_list.append(data)
    # cv2.imshow("Image", img_arr)
    # # cv2.imshow("Image Region", imgregion) #mask
    # cv2.waitKey(0)

# r = requests.get(img_url.format(2))
# img = Image.open(BytesIO(r.content))
# img = np.asarray(img)
# print(img.shape)
# count_in_img(img, 1)


In [ ]:
for i in range(1,11):
    
    r = requests.get(img_url.format(i))
    img = Image.open(BytesIO(r.content))
    img = np.asarray(img)
    # cv2.imshow("Img",img)
    # cv2.waitKey(2000)
    count_in_img(img,i)

df = pd.DataFrame(final_list[0])

for i in range(1, len(final_list)):
    temp_df = pd.DataFrame(final_list[i])
    df = pd.concat([df, temp_df])


0: 352x640 7 persons, 5.7ms
Speed: 1.0ms preprocess, 5.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 8 persons, 1 bench, 11.2ms
Speed: 1.2ms preprocess, 11.2ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 3 persons, 1 umbrella, 10.4ms
Speed: 1.5ms preprocess, 10.4ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 13 persons, 11.1ms
Speed: 2.0ms preprocess, 11.1ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 12 persons, 1 bicycle, 9.7ms
Speed: 1.5ms preprocess, 9.7ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 9 persons, 11.2ms
Speed: 1.4ms preprocess, 11.2ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 11 persons, 12.7ms
Speed: 1.3ms preprocess, 12.7ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 352x640 15 persons, 3 horses, 10.3ms
Speed: 2.3ms preprocess, 10.3ms inferen

In [8]:
df

,image,bicycle,car,person,umbrella,motorbike,handbag
0,1,0,0,7,0,0,0
0,2,0,0,8,0,0,0
0,3,0,0,3,1,0,0
0,4,0,0,13,0,0,0
0,5,1,0,12,0,0,0
0,6,0,0,9,0,0,0
0,7,0,0,11,0,0,0
0,8,0,0,15,0,0,0
0,9,0,0,14,0,0,0
0,10,0,0,15,0,0,0
